In [1]:
import ksig_cpu as ksig
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [2]:
from scipy.io import arff

In [3]:
data_train = arff.loadarff(open('ucwavegesture/UWaveGestureLibraryAll_TRAIN.arff'))
data_test = arff.loadarff(open('ucwavegesture/UWaveGestureLibraryAll_TEST.arff'))


In [4]:
df_train = pd.DataFrame(data_train[0]).to_numpy(dtype=float)
df_test = pd.DataFrame(data_test[0]).to_numpy(dtype=float)

X_train = df_train[:, :945]
y_train  = df_train[:, 945]

X_test = df_test[:, :945]
y_test  = df_test[:, 945]

In [5]:
X_train = X_train.reshape(len(X_train), 3, -1).transpose(0,2,1)
X_test = X_test.reshape(len(X_test), 3, -1).transpose(0,2,1)

scaler_x = StandardScaler()
scaler_y = StandardScaler()
scaler_z = StandardScaler()

X_train[:,:,0] = scaler_x.fit_transform(X_train[:,:,0])
X_train[:,:,1] = scaler_y.fit_transform(X_train[:,:,1])
X_train[:,:,2] = scaler_z.fit_transform(X_train[:,:,2])

X_test[:,:,0] = scaler_x.transform(X_test[:,:,0])
X_test[:,:,1] = scaler_y.transform(X_test[:,:,1])
X_test[:,:,2] = scaler_z.transform(X_test[:,:,2])

X_test = X_test[:700]

In [6]:
X_train.shape

(896, 315, 3)

In [7]:
n_levels = 3
n_order = 1
n_components = 8

In [8]:
static_kernel = ksig.static.kernels.RBFKernel()
proj = ksig.projections.CountSketchRandomProjection(n_components=8)
static_feat = ksig.static.features.NystroemFeatures(static_kernel, n_components=15)

lr_sig_kernel = ksig.kernels.LowRankSignatureKernel(order=n_order, n_levels=n_levels, static_features=static_feat)
lr_sig_kernel.fit(X_train)

LowRankSignatureKernel(n_levels=3, order=1,
                       static_features=NystroemFeatures(n_components=15,
                                                        static_kernel=RBFKernel()))

In [9]:
K_XX = lr_sig_kernel(X_train)

In [10]:
from sklearn.svm import SVC

In [11]:
clf = SVC(C=1., kernel='precomputed')
clf.fit(K_XX, y_train)
kernel_test = lr_sig_kernel(X_test, X_train)
y_pred = clf.predict(kernel_test)

In [12]:
acc = ( ((y_pred == y_test[:700]).sum()/len(y_pred)) )

In [13]:
acc

0.8957142857142857

In [ ]:
acc = []

for theta in [0.1, 1., 10., 100.] :
    for gamma in [0.1, 1., 10., 100.] :
        static_kernel = ksig.static.kernels.RBFKernel()
        proj = ksig.projections.CountSketchRandomProjection(n_components=8)
        static_feat = ksig.static.features.NystroemFeatures(static_kernel, n_components=15)

        lr_sig_kernel = ksig.kernels.LowRankSignatureKernel(order=n_order, n_levels=n_levels, static_features=static_feat)
        lr_sig_kernel.fit(X_train)
        for C in [0.1, 1., 10., 100.] :
            K_XX = lr_sig_kernel(X_train)
            clf = SVC(C=1., kernel='precomputed')
            clf.fit(K_XX, y_train)
            kernel_test = lr_sig_kernel(X_test, X_train)
            y_pred = clf.predict(kernel_test)
            a = ( ((y_pred == y_test[:700]).sum()/len(y_pred)) )
            acc.append(a)
            